In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

root_pth = "/home/user/data/THU-timeseries"
# root_pth = "/mnt/ExtraDisk/wcx/research/THU-timeseries"
all_data_pth = [
    # "dataset/MFRED_clean.csv",
    # "ETT-small/ETTh1.csv",
    # "ETT-small/ETTh2.csv",
    # "ETT-small/ETTm1.csv",
    # "ETT-small/ETTm2.csv",
    # 'exchange_rate/exchange_rate.csv',
    # 'illness/national_illness.csv',
    # 'weather/weather.csv',
    # 'traffic/traffic.csv',
    'electricity/electricity.csv',
]
for data_pth in all_data_pth:
    df = pd.read_csv(os.path.join(root_pth, data_pth), index_col=0, parse_dates=True)
    n_timestep = len(df)
    num_train = int(n_timestep * 0.7)
    df = np.lib.stride_tricks.sliding_window_view(df['OT'].values.flatten(), 96*2)
    df = (df - df.mean(axis=1, keepdims=True))/df.std(axis=1, keepdims=True)
    print(df.std())
    
    # sns.jointplot(df, x=0, y=1)
        
    # scaler = StandardScaler()
    # scaler.fit(df[:num_train])
    # df[df.columns] = scaler.transform(df)
    # df.index = pd.date_range('2019-01-01', periods=len(df), freq='5min')
        
    # df.to_csv(os.path.join(root_pth, 'MFRED_scaled.csv'))
    # long_df = []
    # for c in df.columns.tolist():
    #     temp_df = df[[c]].copy()
    #     temp_df['unique_id'] = c
    #     temp_df = temp_df.reset_index()
    #     temp_df = temp_df.rename(columns={c:'y','date':'ds'})
    #     if data_pth.__contains__('ETTh'):
    #         num_train = 12 * 30 * 24
    #         num_test = 4 * 30 * 24
    #         num_vali = 4 * 30 * 24
    #         temp_df = temp_df[:num_train+num_vali+num_test]
    #     elif data_pth.__contains__('ETTm'):
    #         num_train = 12 * 30 * 24 * 4
    #         num_test = 4 * 30 * 24 * 4
    #         num_vali = 4 * 30 * 24 * 4
    #         temp_df = temp_df[:num_train+num_vali+num_test]
    #     long_df.append(temp_df)
    # long_df = pd.concat(long_df)
    # long_df = long_df.reset_index(drop=True)
    # long_df.to_csv(os.path.join(root_pth, data_pth[:-4]+'_NF.csv'), index=False)
    # break

0.9999999999999996


In [ ]:
import pandas as pd
from datasetsforecast.long_horizon import LongHorizon, LongHorizonInfo
import logging
import os
import argparse
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from neuralforecast import NeuralForecast
from neuralforecast.models import (
    NHITS,
    DLinear,
    PatchTST,
    TimesNet,
    # iTransformer,
    # TFT,
    Autoformer,
    # FEDformer,
    # LSTM,
    # MLP,
    # NBEATSx,
    # DeepAR,
)
from neuralforecast.losses.numpy import mse, mae
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss
from neuralforecast.losses.numpy import mqloss, mse, mae
from src.utils.metrics import calculate_metrics, get_bench_metrics
import pickle

# Change this to your own data to try the model
print(LongHorizonInfo['ECL'].test_size)
Y_df, _, _ = LongHorizon.load(directory='/home/user/data/NF_longterm', group='ECL')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

# For this excercise we are going to take 960 timestamps as validation and test
n_time = len(Y_df.ds.unique())
num_train = int(n_time * 0.7)
num_test = int(n_time * 0.2)
print(num_test)
num_vali = n_time - num_train - num_test
# num_train = 12 * 30 * 24
# num_test = 4 * 30 * 24
# num_vali = 4 * 30 * 24
        
# val_size = 96*10
# test_size = 96*10

Y_df = Y_df[Y_df['unique_id'] == 'OT']


In [ ]:
y_true = Y_hat_df["y"].values.reshape(-1, pred_len, 1)
print(y_true.shape)
y_true[0][:10]

In [ ]:
# SAVE METRICS
ds = [
    "ETTh1",
    "ETTh2",
    "ETTm1",
    "ETTm2",
    "ECL",
    "Exchange",
    "TrafficL",
    "Weather",
]
pred_len = [96, 192, 336, 720]
save_dir = "/mnt/ExtraDisk/wcx/research/benchmarks"
all_df = []
for d in ds:
    ds_df = []
    for pl in pred_len:
        result_path = os.path.join(save_dir, f"{d}_96_{pl}_U","results.csv")
        df = pd.read_csv(result_path, index_col=0)
        df.index.name = 'model'
        df = df.reset_index()
        df = df.groupby('model').mean()
        df = df.drop(columns=['MAE', 'iter'])
        df = df.stack()
        df = pd.DataFrame(df, columns=[pl]).transpose()
        ds_df.append(df)
    ds_df = pd.concat(ds_df)
    ds_df.index.name = 'pred_len'
    ds_df['dataset'] = d
    ds_df = ds_df.reset_index()
    ds_df = ds_df.set_index(['dataset','pred_len'])
    all_df.append(ds_df)
all_df = pd.concat(all_df)
all_df.to_csv(os.path.join(save_dir, 'bench_result.csv'))
# print(all_df.to_latex(float_format="{:.3f}".format))

In [ ]:
import pandas as pd
from datasetsforecast.long_horizon import LongHorizon
import os
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np


ds = {
        "ETTh1": "ETTh1",
        "ETTh2": "ETTh2",
        "ETTm1": "ETTm1",
        "ETTm2": "ETTm2",
        "ECL": "electricity",
        "Exchange": "exchange_rate",
        "TrafficL": "traffic",
        "Weather": "weather",
        "MFRED": "mfred",
    }

for d in ds:
    print(d)
    if d=='Weather':
        data_dir_nf = '/mnt/ExtraDisk/wcx/research'
        dataset_nf = d
        data_dir_thu = '/mnt/ExtraDisk/wcx/research/THU-timeseries'
        dataset_thu = f"{'ETT-small' if ds[d].__contains__('ETT') else ds[d]}/{ds[d]}.csv"
        csv_thu = os.path.join(data_dir_thu, dataset_thu)
        
        # THU
        df_thu = pd.read_csv(csv_thu, index_col=0, parse_dates=True)[['OT']]
        if d.__contains__("ETTh"):
            num_train = 12 * 30 * 24
            # num_test = 4 * 30 * 24
            # num_vali = 4 * 30 * 24
        elif d.__contains__("ETTm"):
            num_train = 12 * 30 * 24 * 4
            # num_test = 4 * 30 * 24 * 4
            # num_vali = 4 * 30 * 24 * 4
        else:
            num_train = int(len(df_thu) * 0.7)
            # num_test = int(len(Y_df) * 0.2)
            # num_vali = len(Y_df) - num_train - num_test
        scaler = StandardScaler()
        scaler.fit(df_thu.values[:num_train])
        df_thu_value = scaler.transform(df_thu.values)
        
            
        # NF
        Y_df, _, _ = LongHorizon.load(directory=data_dir_nf, group=dataset_nf)
        Y_df["ds"] = pd.to_datetime(Y_df["ds"])
        df_nf = Y_df[Y_df["unique_id"] == "OT"]
        df_nf = df_nf.drop(columns=['unique_id'])
        df_nf = df_nf.set_index('ds')
        
        if df_thu_value.shape != df_nf.values.shape:
            print("shape is not the same")
            print(f'{d}')
            print(df_thu_value.shape)
            print(df_nf.values.shape)
            max_len = min(len(df_thu_value), len(df_nf.values))
        else:
            max_len = len(df_thu_value)
        
        close = np.allclose(df_thu_value.flatten()[:max_len], df_nf.values.flatten()[:max_len])
        close = np.allclose(df_thu_value.flatten()[-10000:], df_nf.values.flatten()[-10000:])
        print(close)
        if not close:
            # plt.plot(df_thu_value.flatten()[-10000:])
            # plt.plot(df_nf.values.flatten()[-10000:])
            plt.scatter(df_thu_value.flatten()[-10000:], df_nf.values.flatten()[-10000:])
        print('----'*10)
    # fig, ax = plt.subplots()
    # ax.scatter(df_thu_value.flatten()[:max_len], df_nf.values.flatten()[:max_len])

In [ ]:
import pandas as pd
from datasetsforecast.long_horizon import LongHorizon
import os
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
from neuralforecast.losses.numpy import mse, mqloss

# # SAVE METRICS
ds = [
    "ETTh1",
    "ETTh2",
    "ETTm1",
    "ETTm2",
    "ECL",
    "Exchange",
    "TrafficL",
    "Weather",
]
quantiles = [0.05 * i for i in range(19)]
# pred_len = [336]
pred_len = [96, 192, 336, 720]
save_dir = "/mnt/ExtraDisk/wcx/research/benchmarks"
all_df = []
for d in ds:
    # if d != 'TrafficL':
    #     continue
    ds_df = []
    for pl in pred_len:
        label_path = os.path.join(save_dir, f"{d}_96_{pl}_U", "true.npy")
        pred_path = os.path.join(save_dir, f"{d}_96_{pl}_U", "pred_PatchTST_0.npy")
        y_true = np.load(label_path)
        y_pred = np.load(pred_path)
        y_true_fft = np.fft.rfft(y_true[0].flatten(), norm="ortho")
        y_true_fft[0] = 0
        y_tr_fft_abs = np.abs(y_true_fft)
        theta = np.arctan2(y_true_fft.imag, y_true_fft.real)
        phi_r_int = np.where(
            np.isinf(y_tr_fft_abs),
            np.ones_like(y_tr_fft_abs),
            ((y_tr_fft_abs - 1) / (y_tr_fft_abs + 1)),
        )
        phi = np.arcsin(phi_r_int)
        # print(theta[0])
        print(theta[-1])
        # y_true = y_true[::pl].flatten()
        fig, ax = plt.subplots()
        ax.plot(theta)
        # ax.plot(y_pred[333, ..., 9].flatten())

        ax.set_title(d)
        # plt.plot(y_true[126].flatten())
        # plt.plot(y_pred[126, ..., 9].flatten())
        break
    # break
#         df.index.name = 'model'
#         df = df.reset_index()
#         df = df.groupby('model').mean()
#         df = df.drop(columns=['MAE', 'iter'])
#         df = df.stack()
#         df = pd.DataFrame(df, columns=[pl]).transpose()
#         ds_df.append(df)
#     ds_df = pd.concat(ds_df)
#     ds_df.index.name = 'pred_len'
#     ds_df['dataset'] = d
#     ds_df = ds_df.reset_index()
#     ds_df = ds_df.set_index(['dataset','pred_len'])
#     all_df.append(ds_df)
# all_df = pd.concat(all_df)
# all_df.to_csv(os.path.join(save_dir, 'bench_result.csv'))
# print(all_df.to_latex(float_format="{:.3f}".format))

In [ ]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import QuantileRegressor

ds = {
    "ETTh1": "etth1",
    # "ETTh2": "etth2",
    # "ETTm1": "ettm1",
    # "ETTm2": "ettm2",
    # "ECL": "electricity",
    # "Exchange": "exchange_rate",
    # "TrafficL": "traffic",
    # "Weather": "weather",
    # "MFRED": "mfred",
}
# pred_len = [288, 432, 576]
pred_len = [96]
# pred_len = [96, 192, 336, 720]
save_dir = "/home/user/data/FrequencyDiffusion/savings"
save_dir_bench = "/home/user/data/NF_benchmark"
# save_dir = "/mnt/ExtraDisk/wcx/research/FrequencyDiffusion/savings"
all_df = []

for d in ds:
    real_d = ds[d]
    ds_df = []
    for pl in pred_len:
        print(d, pl)
        result_path = os.path.join(save_dir, f"{real_d}_{pl}_S", "test_dl.pt")
        test_dl = torch.load(result_path)
        y_real = []
        x = []
        for b in test_dl:
            x.append(b["observed_data"].cpu().numpy())
            y_real.append(b["future_data"].cpu().numpy())
        y_real = np.concatenate(y_real)
        print(y_real.shape)
        print(y_real.flatten()[:10])
        print(y_real.flatten()[-10:])
        # print(y_real[-1])
        



    #     x_norm = (x - x.mean(axis=1, keepdims=True))/x.std(axis=1, keepdims=True)
    #     y_real_norm = (y_real - x.mean(axis=1, keepdims=True))/x.std(axis=1, keepdims=True)

    #     fig, axs = plt.subplots(3,3)
    #     axs = axs.flatten()
    #     choose = np.random.randint(0, len(x), size=len(axs))
    #     for i in range(len(axs)):
    #         # axs[i].plot(range(0, x.shape[1]), x[choose[i], :, 0])
    #         # axs[i].plot(range(0, x.shape[1]), np.mean(x[choose[i], :, 0], axis=0, keepdims=True).repeat(x.shape[1]), ls='--', c='grey')
    #         # axs[i].plot(range(x.shape[1], x.shape[1]+y_real.shape[1]), y_real[choose[i], :, 0])
    #         # axs[i].plot(range(x.shape[1], x.shape[1]+y_real.shape[1]), np.mean(y_real[choose[i], :, 0], axis=0, keepdims=True).repeat(y_real.shape[1]), ls='--', c='grey')

    #         axs[i].plot(range(0, x_norm.shape[1]), x_norm[choose[i], :, 0])
    #         axs[i].plot(range(0, x_norm.shape[1]), np.mean(x_norm[choose[i], :, 0], axis=0, keepdims=True).repeat(x.shape[1]), ls='--', c='grey')
    #         axs[i].plot(range(x.shape[1], x.shape[1]+y_real.shape[1]), y_real_norm[choose[i], :, 0])
    #         axs[i].plot(range(x.shape[1], x.shape[1]+y_real.shape[1]), np.mean(y_real_norm[choose[i], :, 0], axis=0, keepdims=True).repeat(y_real_norm.shape[1]), ls='--', c='grey')
    #     # fig.suptitle(f'{d}-96-{pl}')
    #     fig.tight_layout()
    #     break
    # break
    # fig.savefig(f'../assets/{d}-96-{pl}.png')

    # bench_path = os.path.join(save_dir_bench, f"{d}_96_{pl}_S", "true.npy")
    # y_real_bench = np.load(bench_path)
    # print(np.allclose(y_real, y_real_bench))
    # # assert np.allclose(y_real, y_real_bench)

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('../assets/bj_aq_online.csv')
df = df.dropna(axis=1, how='all')
for i, temp_df in df.groupby('station_id'):
    temp_df = temp_df.drop(columns=['id', 'station_id'])
    temp_df['time'] = pd.to_datetime(temp_df['time'])
    temp_df = temp_df.sort_values('time')
    temp_df = temp_df.set_index('time')
    temp_df[:100].plot()
    

* FORECAST RESULTS

In [9]:
import pandas as pd

bs = 128
# model_name = f"MADtime_pl_Full_SETKS_FreqDoi_bs{bs}"
# model_name = f"MADtime_pl_Full_SETKS_bs{bs}"
# model_name = f"MADtime_pl_FactOnly_FreqDoi_bs{64}"
# model_name = f"MADtime_pl_FactOnly_SETKS_FreqDoi_CFG_puncond0.8_bs{bs}_fcst"
# model_name = "MADtime_FactOnly_SETKS_learnmean_freqdenoise_bs64_fast"
# model_name = "MADtime_FactOnly_SETKS_learnmean_freqdenoise_puncond0.5_bs64_fcst"
model_name = "MADfreq_FactOnly_puncond0.5_bs64_fcst"
# model_name = "MADtime_FactOnly_SETKS_learnmean_freqdenoise_bs64_new"

# * BEST FCST
# model_name = "best_MADtime_pl_FactOnly_FreqDoi"

datasets = ['ECL','ETTh2','ETTm2','Exchange','Weather']

new_df = pd.read_csv(f"../assets/{model_name}.csv")
# new_df = pd.read_csv('../assets/MADtime_pl_doublenorm_FreqDenoise.csv')[['MSE']]
# print(new_df)
new_df = new_df.rename(columns={"MSE": model_name})
new_df = new_df.drop(columns=["CRPS", "method"])
df = pd.read_csv("../assets/results_MSE.csv", delimiter="\t")
# print(df)

df = pd.merge(df, new_df)

# df = pd.concat([df, new_df], axis=1)
df = df.groupby("dataset").mean()
# # df
# df = df[['Dlinear', 'PatchTST', 'CSDI','D3VAE', 'SSSD', 'TMDM', 'mr-diff','MADtime_freqdenoise_bs128']]
df = df[
    [
        "Dlinear",
        "PatchTST",
        "CSDI",
        "SSSD",
        "D3VAE",
        "TMDM",
        "mr-diff",
        model_name,
    ]
]
# df = df[['Dlinear', 'PatchTST', 'CSDI', 'SSSD', 'D3VAE','TMDM', 'mr-diff','MADfreq_pl_doublenorm_dp0.3','MADtime_freqdenoise']]
df = df.T
# df = df.drop(columns=['MFRED']).round(4)
# df = df.drop(columns=["MFRED", "ETTh1", "ETTm1"]).round(4)
df = df.round(4)
df = pd.concat([df, df.rank().mean(axis=1).round(2)], axis=1)
df

# print(df.to_latex())

,ECL,ETTh2,ETTm2,Exchange,traffic,weather,0
Dlinear,0.3926,0.2239,0.1255,0.3681,0.3226,0.1913,2.67
PatchTST,0.4116,0.2026,0.1219,0.5154,0.1784,0.1889,2.00
CSDI,0.4581,0.2571,2.1230,1.2557,0.4991,0.1938,5.50
SSSD,1.0257,0.7201,0.8936,2.9004,1.9662,0.6905,7.17
D3VAE,0.8450,1.3961,3.3449,2.1086,6.3583,1.5461,7.67
TMDM,0.4071,0.2508,0.1789,0.7885,0.1805,0.2209,4.33
mr-diff,0.5287,0.2172,0.1700,0.4801,0.2471,0.2078,3.83
MADfreq_FactOnly_puncond0.5_bs64_fcst,0.3692,0.2101,0.1273,0.4110,0.2525,0.2132,2.83


* Refinement results

In [7]:
import pandas as pd

bs = 64

# model_name = f"MADtime_pl_FactOnly_SETKS_FreqDoi_CFG_bs{bs}_refine"
# model_name = f"MADtime_pl_FactOnly_SETKS_FreqDoi_CFG_puncond0.8_bs{bs}_refine"
# model_name = f"MADtime_FactOnly_SETKS_learnmean_freqdenoise_puncond0.5_bs{bs}_refine"
model_name = "MADfreq_puncond0.5_bs64_refine"
# model_name = "MADfreq_FactOnly_puncond0.5_bs64_refine"
# assets/MADfreq_FactOnly_puncond0.5_bs64_refine.csv

new_df = pd.read_csv(f"../assets/{model_name}.csv")
# new_df = pd.read_csv('../assets/MADtime_pl_doublenorm_FreqDenoise.csv')[['MSE']]
# print(new_df)
new_df = new_df.rename(columns={"MSE": model_name})
new_df = new_df.drop(columns=["CRPS", "method"])
df = pd.read_csv("../assets/results_MSE.csv", delimiter="\t")
# print(df)

df = pd.merge(df, new_df)

# df = pd.concat([df, new_df], axis=1)
df = df.groupby("dataset").mean()
# # df
# df = df[['Dlinear', 'PatchTST', 'CSDI','D3VAE', 'SSSD', 'TMDM', 'mr-diff','MADtime_freqdenoise_bs128']]
df = df[
    [
        "PatchTST",
        model_name,
    ]
]
# df = df[['Dlinear', 'PatchTST', 'CSDI', 'SSSD', 'D3VAE','TMDM', 'mr-diff','MADfreq_pl_doublenorm_dp0.3','MADtime_freqdenoise']]
df = df.T
# df = df.drop(columns=['MFRED']).round(4)
# df = df.drop(columns=["MFRED", "ETTh1", "ETTm1"]).round(4)
df = df.round(4)
df = pd.concat([df, df.rank().mean(axis=1).round(2)], axis=1)
df

# print(df.to_latex())

,ECL,ETTh2,ETTm2,Exchange,traffic,weather,0
PatchTST,0.4116,0.2026,0.1219,0.5154,0.1784,0.1889,1.17
MADfreq_puncond0.5_bs64_refine,0.8915,0.2586,0.1311,0.4499,1.8220,0.1904,1.83


In [2]:
import pandas as pd

bs = 64
# model_name = f"MADtime_pl_Full_SETKS_FreqDoi_bs{bs}"
# model_name = f"MADtime_pl_Full_SETKS_bs{bs}"
# model_name = f"MADtime_pl_FactOnly_FreqDoi_bs{bs}"
# model_name = "best_MADtime_pl_FactOnly_FreqDoi"
# model_name = "MADtime_pl_doublenorm_FreqDenoise"
# model_name = f"MADfreq_naive_bs{bs}"
# model_name = f"MADfreq_learnmean_bs{bs}_new"
model_name = "MADfreq_FactOnly_learnmean_bs64_new"
# model_name = "MADtime_FactOnly_SETKS_learnmean_freqdenoise_bs64_new"


new_df = pd.read_csv(f"../assets/{model_name}.csv")
# new_df = pd.read_csv('../assets/.csv')[['MSE']]
# print(new_df)
new_df = new_df.rename(columns={"CRPS": model_name})
new_df = new_df.drop(columns=["MSE", "method"])
df = pd.read_csv("../assets/results_CRPS.csv", delimiter="\t")
# print(df)

df = pd.merge(df, new_df)

# df = pd.concat([df, new_df], axis=1)
df = df.groupby("dataset").mean()
df = df[
    [
        "CSDI",
        "SSSD",
        "D3VAE",
        "TMDM",
        "mr-diff",
        # "MADtime_pl_doublenorm",
        model_name,
    ]
]
# df = df[['Dlinear', 'PatchTST', 'CSDI', 'SSSD', 'D3VAE','TMDM', 'mr-diff','MADfreq_pl_doublenorm_dp0.3','MADtime_freqdenoise']]
df = df.T
# df = df.drop(columns=['MFRED']).round(4)
# df = df.drop(columns=["MFRED", "ETTh1", "ETTm1"]).round(4)
df = df.round(4)
df = pd.concat([df, df.rank().mean(axis=1).round(2)], axis=1)
df

# print(df.to_latex())

,ECL,ETTh2,ETTm2,Exchange,traffic,weather,0
CSDI,0.1939,0.1638,0.4720,0.3028,0.1883,0.1261,3.33
SSSD,0.3216,0.3108,0.3565,0.6743,0.4579,0.3129,5.17
D3VAE,0.3111,0.4173,0.6497,0.5380,0.8314,0.4497,5.67
TMDM,0.1881,0.1591,0.1253,0.2959,0.1076,0.1380,2.00
mr-diff,0.2357,0.1647,0.1293,0.2117,0.1453,0.1506,3.17
MADfreq_FactOnly_learnmean_bs64_new,0.1891,0.1465,0.1101,0.2392,0.1163,0.1375,1.67
